## IMPORT LIBRARIES

In [4]:
import pandas as pd
import os

## LOAD CSV DATASETS
we are going to use dictionary to load them faster instead of doing them one by one

In [5]:
# Define a dictionary of file paths
file_paths = {
    "Facebook": "../data/raw/Post Performance (Stanbic IBTC) January 1, 2013 - July 13, 2023_Facebook.csv",
    "Instagram": "../data/raw/Post Performance (Stanbic IBTC) January 1, 2013 - July 13, 2023_Instagram.csv",
    "LinkedIn": "../data/raw/Post Performance (Stanbic IBTC) January 1, 2013 - July 13, 2023_LinkedIn.csv",
    "Twitter": "../data/raw/Post Performance (Stanbic IBTC) January 1, 2013 - July 13, 2023_Twitter.csv"
}

# Create an empty dictionary to store DataFrames
dataframes = {}

# Load the datasets into pandas DataFrames using a loop
for platform, file_path in file_paths.items():
    dataframes[platform] = pd.read_csv(file_path,dtype='object')


## CLEAN DATA
Data Cleaning (customizing the cleaning steps for each platform)

Steps includes
1. Columns are renamed according to the specified mapping dictionary.
2. Commas and percentage symbols are removed from columns where needed.
3. The "Engagements" and "Impressions" columns are converted to numeric, handling non-numeric values gracefully by converting them to NaN.
4. Missing values in the "post_link_clicks" column are filled with zeros.
5. Data types for specific columns are defined and enforced.
6. Unwanted columns are removed from the DataFrame.
7. Rows with missing values in "impressions" and "organic_impressions" columns are dropped.
8. The code now checks if the cleaned data file already exists and overwrites it if it does.

In [8]:
# Define the directory paths for cleaned data
cleaned_data_dir = "../data/interim"

# Create the directories if they don't exist
os.makedirs(cleaned_data_dir, exist_ok=True)

# Columns to Rename
# Columns to Rename
columns_to_rename = {
    "Date": "date",
    "Impressions": "impressions",
    "Organic Impressions": "organic_impressions",
    "Viral Impressions": "viral_impressions",
    "Non-viral Impressions": "non_viral_impressions",
    "Paid Impressions": "paid_impressions",
    "Engagement Rate (per Impression)": "engagement_rate_per_impression",
    "Engagement Rate (per Reach)": "engagement_rate_per_reach",
    "Engagements": "engagements",
    "Reactions": "reactions",
    "Comments": "comments",
    "Shares": "shares",
    "Likes": "likes",
    "Dislikes": "dislikes",
    "Love Reactions": "love_reactions",
    "Haha Reactions": "haha_reactions",
    "Wow Reactions": "wow_reactions",
    "Sad Reactions": "sad_reactions",
    "Angry Reactions": "angry_reactions",
    "Saves": "saves",
    "Click-Through Rate": "click_through_rate",
    "Post Link Clicks": "post_link_clicks",
    "Other Post Clicks": "other_post_clicks",
    "Post Clicks (All)": "post_clicks_all",
    "Post Media Clicks": "post_media_clicks",
    "Post Hashtag Clicks": "post_hashtag_clicks",
    "Post Detail Expand Clicks": "post_detail_expand_clicks",
    "Profile Clicks": "profile_clicks",
    "Post Photo View Clicks": "post_photo_view_clicks",
    "Post Video Play Clicks": "post_video_play_clicks",
    "Link": "links",
    "Post": "post",
    "Content Type": "content_type",
    "Network": "network"
}
numeric_col = [
    "post_link_clicks",
    "engagement_rate_per_impression",
    "engagement_rate_per_reach",
    "reactions",
    "engagements",
    "likes",
    "dislikes",
    "impressions",
    "non_viral_impressions",
    "engagement_rate_per_impression", 
    "organic_impressions",
    "love_reactions",
    "haha_reactions",
    "wow_reactions",
    "sad_reactions",
    "angry_reactions",
    "saves",
    "comments",
    "other_post_clicks",
    "shares",
    "post_clicks_all",
    "click_through_rate",
    "post_link_clicks",
    "reactions",
    "viral_impressions",
    "non_viral_impressions",
    "paid_impressions",
    "post_video_play_clicks",
    "profile_clicks",
    "post_photo_view_clicks",
    "post_detail_expand_clicks",
    "post_hashtag_clicks",
    "post_media_clicks",
]
# Columns to Delete
columns_to_delete = [
    "Post ID",
    "Profile",
    "Sent by",
    "Linked Content",
    "Fan Impressions",
    "Fan Organic Impressions",
    "Fan Paid Impressions",
    "Non-fan Impressions",
    "Non-fan Organic Impressions",
    "Non-fan Paid Impressions",
    "Non-viral Reach",
    "Paid Reach",
    "Fan Reach",
    "Fan Paid Reach",
    "Potential Reach",
    "Other Engagement Metrics",
    "Post Video Play Clicks"  # Remove "Post Video Play Clicks" column
]

columns_with_symbols = []

cleaned_dataframes={}
# Iterate through the original DataFrames
for platform, df in dataframes.items():
    # Rename columns
    df.rename(columns=columns_to_rename, inplace=True)
    
    # Remove commas and percentage from columns where needed (e.g., "impressions")
    for col in numeric_col:
        # Convert the columns to numeric
        df[col] = pd.to_numeric(df[col], errors='coerce')  # 'coerce' will convert non-numeric values to NaN
        if not df[col].empty and df[col].count() > 0:
            df[col] = df[col].replace({',': '', '%': ''}, regex=True)       
        df[col].fillna(0, inplace=True)




    # Define data types for specific columns
    data_types = {
        "date": "datetime64[ns]",  # Assuming "date" column contains date values
        "engagement_rate_per_impression": float,
        "engagement_rate_per_reach": float,
        "reactions": int,
        "non_viral_impressions":int,
        "comments": int,
        "paid_impressions":int,
        "viral_impressions":int,
        "shares": int,
        "click_through_rate": float,
        "post_link_clicks": int,
        "other_post_clicks": int,
        "reactions":int
    }
    # Define the date format for the "date" column
    date_format = "%m/%d/%Y %I:%M %p"
    # # Convert the "date" column to datetime with the specified format
    df['date'] = pd.to_datetime(df['date'], format=date_format, errors='coerce')

    # Convert columns to specified data types
    df = df.astype(data_types, errors='ignore')

    # Keep selected columns
    df = df[columns_to_rename.values()].copy()
    
    # Remove unwanted columns
    df.drop(columns=columns_to_delete, inplace=True, errors="ignore")

    # Removing rows with missing values in specific columns
    df.dropna(subset=["impressions", "organic_impressions","post","links"], inplace=True)

    # Sort the DataFrame by the "date" column
    df.sort_values(by="date", inplace=True)

    # Save the cleaned DataFrame to the appropriate directory
    cleaned_data_file = f"{platform}_cleaned.csv"
    cleaned_data_path = os.path.join(cleaned_data_dir, cleaned_data_file)
    
    # Check if the file already exists and overwrite it
    if os.path.exists(cleaned_data_path):
        df.to_csv(cleaned_data_path, index=False, mode='w')
    else:
        df.to_csv(cleaned_data_path, index=False)

    # Store the cleaned DataFrame in the dictionary
    cleaned_dataframes[platform] = df

print("Data cleaning and saving complete.")



Data cleaning and saving complete.
